In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
moist = pd.read_csv("sephora_moisturizers.csv")
cleansers = pd.read_csv("sephora_cleansers.csv")

In [3]:
moist = moist.drop(moist.columns[0], axis=1)
cleansers = cleansers.drop(cleansers.columns[0], axis=1)

## Combine the datasets and shuffle the rows

In [4]:
full_data = pd.concat([moist, cleansers], ignore_index=True).sample(frac=1).reset_index(drop=True)

In [5]:
full_data

,product_url,overall_ratings,brand,skin_type,skincare_concerns,is_vegan,is_cruelty_free,award_wins,is_clean_product,clinical_results,ingredients,price,highlights,review_count,sizing_options_count,image_count,video_count
0,https://www.sephora.com/product/ultra-repair-h...,4.5,First Aid Beauty,"Normal, Dry, and Combination","Fine Lines and Wrinkles, Dryness, and Loss of ...",False,True,0,True,In a 4-week independent clinical study with 38...,"-Colloidal Oatmeal: Calms and relieves dry, di...",40.0,"Fragrance Free,Cruelty-Free,Best for Dry, Comb...",534,1,4,0
1,https://www.sephora.com/product/milk-makeup-ve...,4.5,MILK MAKEUP,Normal and Dry,NaN,False,True,0,False,NaN,-Desert Milk™: Hydrates and helps skin stay mo...,15.0,"Clean at Sephora,Best for Combination Skin",817,1,3,0
2,https://www.sephora.com/product/sephora-collec...,4.0,SEPHORA COLLECTION,"Normal, Dry, Combination, and Oily","Pores, Dullness and Uneven Texture, Oiliness",True,True,0,True,NaN,-Coconut Milk: Contains vitamins and sugars t...,4.0,"Clean at Sephora,Vegan",104,1,0,0
3,https://www.sephora.com/product/invisimatte-bl...,4.5,FENTY BEAUTY by Rihanna,NaN,NaN,False,False,0,False,NaN,Suggested Usage:-Blot on the fly as needed thr...,10.0,NaN,18,1,2,1
4,https://www.sephora.com/product/summer-fridays...,4.5,Summer Fridays,"Normal, Dry, Combination, and Oily","Dryness, Dullness, and Uneven Texture",True,True,0,True,NaN,-Hyaluronic Acid Complex: Penetrates skin’s su...,42.0,"Best for Dry, Combo, Normal Skin,Good for: Dry...",176,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,https://www.sephora.com/product/epidermal-re-t...,4.5,Kiehl's Since 1851,NaN,NaN,False,False,0,False,NaN,-Micronized Shells (Diatomatious Earth): A nat...,41.0,NaN,66,1,3,0
1172,https://www.sephora.com/product/coconut-rose-t...,4.5,Kopari,"Normal, Dry, Combination, and Oily","Redness, Dryness, and Pores",True,True,0,True,NaN,-Witch Hazel: This natural toner calms and res...,25.0,"Clean at Sephora,Cruelty-Free",194,1,4,1
1173,https://www.sephora.com/product/rose-hibiscus-...,4.0,Herbivore,Normal and Dry,"Dryness, Dullness and Uneven Texture, and Loss...",True,True,0,True,NaN,-Organic Rosewater: Naturally antioxidant-ric...,32.0,"Clean at Sephora,Vegan",55,1,2,0
1174,https://www.sephora.com/product/goop-g-tox-mal...,4.0,goop,"Normal, Dry, Combination, and Oily","Pores, Uneven Texture, and Acne and Blemishes",True,True,0,True,Based on a third-party clinical study of 26 wo...,-Malachite Extract (Rich in Copper): Helps de...,75.0,"Clean at Sephora,Best for Oily, Combo, Normal ...",11,1,3,1


## Cleaning the columns

#### Cleaning skin_type

In [6]:
# First we see how many different possible skin_types there are.
possible_skin_types = set()
nan_counter = 0
for i in full_data["skin_type"]:
    # replace nan with the string "Empty" since 436 out of 1176 data rows has Nan (we treat it as an individual type.)
    if not isinstance(i, str):
        i = "Empty"
        nan_counter += 1
    # replace "," and "and" with space.
    cleaned = i.replace("and", " ")
    cleaned = cleaned.replace(".", " ")
    cleaned = cleaned.replace(",", " ")
    tokens = cleaned.split()
    possible_skin_types.update(tokens)

print(nan_counter)
possible_skin_types

436


{'Combination', 'Dry', 'Empty', 'Normal', 'Oil', 'Oily', 'Sensitive'}

In [7]:
# For some reason, one particular item has type "Oil" instead of "Oily". We treat these 2 as the same.

# Convert the skin_type column into 5 dummy columns (6 possible skin types, but I use Empty as the reference type) 
# where each column means something like "supports X skin type".
skin_type_cols = pd.DataFrame(columns = ["combination_skin_type", "dry_skin_type", "normal_skin_type", "oily_skin_type", "sensitive_skin_type"])
# populate this new augmented data
for i in full_data["skin_type"]:
    row = [0,0,0,0,0]
    # if is nan, we append a row with all 0s
    if isinstance(i, str):
        if "Combination" in i:
             row[0] = 1
        if "Dry" in i:
            row[1] = 1
        if "Normal" in i:
            row[2] = 1
        if "Oil" in i:
            row[3] = 1
        if "Sensitive" in i:
            row[4] = 1
    skin_type_cols.loc[len(skin_type_cols)] = row

# replace skin_type column with these 5 columns
cleaned_data = full_data.drop("skin_type", axis=1)
cleaned_data = pd.concat([cleaned_data, skin_type_cols], axis=1)

#### Cleaning skincare_concerns

In [8]:
# after splitting into tokens by commas and "and" we're left
# with only 27 possible concerns. Hardcoded the remaining 
# groups that aren't easily identified through splits

# This reduces the groups to 15, and each is unique
# The possible groups are {'wrinkles', 'dryness', 'dark circles', 'uneven tone', 'loss of firmness', 'acne', 'dark spots', 'targets signs of aging', 'puffiness', 'lightweight liquid', 'uneven texture', 'redness', 'oiliness', 'makeup removal', 'pores'}
texture = {"uneven texture", "uneven skin textures", "uneven skin texture",
          "dullness/uneven texture", "dullness"}
wrinkles = {"wrinkles", "fine lines", "fine lines/wrinkles"}
firm = {"loss of firmness/elasticity", "loss of firmness", "elasticity"}
tone = {"uneven skin tone", "uneven skintone"}
spots = {"dark spot", "dark spots"}
acne = {"acne", "acne/blemishes", "blemishes"}
skincare_concerns_cols = pd.DataFrame(columns = ['wrinkles_skincare_concerns', 'dryness_skincare_concerns', 'dark_circles_skincare_concerns', 'uneven_tone_skincare_concerns', 'loss_of_firmness_skincare_concerns', 'acne_skincare_concerns', 'dark_spots_skincare_concerns', 'targets_signs_of_aging_skincare_concerns', 'puffiness_skincare_concerns', 'lightweight_liquid_skincare_concerns', 'uneven_texture_skincare_concerns', 'redness_skincare_concerns', 'oiliness_skincare_concerns', 'makeup_removal_skincare_concerns', 'pores_skincare_concerns'])
for concerns in full_data["skincare_concerns"]:
    row = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    if type(concerns) == str:
        concerns = concerns.split(", ")
        if "and " == concerns[-1][:4]:
            concerns[-1] = concerns[-1][4:]
        for concern in concerns:
            split_concern = concern.split("and")
            for individual_concern in split_concern:
                processed = individual_concern.lower().strip()
                if processed in wrinkles:
                    row[0] = 1
                elif processed == 'dryness':
                    row[1] = 1
                elif processed == "dark circles":
                    row[2] = 1
                elif processed in tone:
                    row[3] = 1
                elif processed in firm:
                    row[4] = 1
                elif processed in acne:
                    row[5] = 1
                elif processed in spots:
                    row[6] = 1
                elif processed == 'targets signs of aging':
                    row[7] = 1
                elif processed == 'puffiness':
                    row[8] = 1
                elif processed == 'lightweight liquid':
                    row[9] = 1
                elif processed in texture:
                    row[10] = 1 
                elif processed == 'redness':
                    row[11] = 1
                elif processed == 'oiliness':
                    row[12] = 1
                elif processed == 'makeup removal':
                    row[13] = 1
                elif processed == 'pores':
                    row[14] = 1
    skincare_concerns_cols.loc[len(skincare_concerns_cols)] = row
    
# replace skincare_concerns column with these 15 columns
cleaned_data = cleaned_data.drop("skincare_concerns", axis=1)
cleaned_data = pd.concat([cleaned_data, skincare_concerns_cols], axis=1)

#### Cleaning clinical_results

In [9]:
cleaned_data["clinical_results"] = 1 - full_data["clinical_results"].isna().astype(int)

#### Turning all the other columns with true/false into 1/0

In [10]:
cleaned_data["is_vegan"] = full_data["is_vegan"].astype(int)
cleaned_data["is_cruelty_free"] = full_data["is_cruelty_free"].astype(int)
cleaned_data["is_clean_product"] = full_data["is_clean_product"].astype(int)

#### Dummy encoding brand

In [11]:
cleaned_data["brand"].value_counts()

CLINIQUE              74
Shiseido              40
SEPHORA COLLECTION    40
Kiehl's Since 1851    38
fresh                 32
                      ..
INC.redible            1
MAKEUP BY MARIO        1
Vitruvi                1
Isle of Paradise       1
Peace Out              1
Name: brand, Length: 133, dtype: int64

In [12]:
brand_dummies = pd.get_dummies(full_data["brand"], prefix="brand")
# using brand AMOREPACIFIC as the reference
brand_dummies = brand_dummies.drop("brand_AMOREPACIFIC", axis=1)
cleaned_data = cleaned_data.drop("brand", axis=1)
cleaned_data = pd.concat([cleaned_data, brand_dummies], axis=1)

#### Removing review_count since we wouldnt have this in actual data.

In [13]:
cleaned_data = cleaned_data.drop(["review_count"], axis=1)

#### Transforming ingredients into binary variable representing if it contains alcohol

In [14]:
# from https://www.everydayhealth.com/smart-skin/alcohol-in-skin-care-is-it-ever-okay/
# 
common_alcohols = ["ethyl", "alcohol", "isopropyl", "methanol", "methyl", "benzyl", "cetyl", "stearyl", "cetearyl", "lanolin", "denat"]
cleaned_ingredients = []
for idx, i in enumerate(cleaned_data['ingredients']):
    if not isinstance(i, str):
        cleaned_ingredients.append(0)
        continue
    else:
        found = False
        for a in common_alcohols:
            if a in i:
                cleaned_ingredients.append(1)
                found = True
                break
        if not found:
            cleaned_ingredients.append(0)

In [15]:
cleaned_data['ingredients'].str.contains("denatured").value_counts()

False    1153
Name: ingredients, dtype: int64

In [16]:
cleaned_data['ingredients'] = cleaned_ingredients
cleaned_data = cleaned_data.rename(columns={'ingredients': 'has_alcohol', 'clinical_results': 'has_clinical_results'})
cleaned_data = cleaned_data.drop(["highlights"], axis=1)

In [17]:
cleaned_data

,product_url,overall_ratings,is_vegan,is_cruelty_free,award_wins,is_clean_product,has_clinical_results,has_alcohol,price,sizing_options_count,...,brand_bareMinerals,brand_belif,brand_fresh,brand_goop,brand_innisfree,brand_lilah b.,brand_philosophy,brand_rms beauty,brand_tarte,brand_trèStiQue
0,https://www.sephora.com/product/ultra-repair-h...,4.5,0,1,0,1,1,1,40.0,1,...,0,0,0,0,0,0,0,0,0,0
1,https://www.sephora.com/product/milk-makeup-ve...,4.5,0,1,0,0,0,1,15.0,1,...,0,0,0,0,0,0,0,0,0,0
2,https://www.sephora.com/product/sephora-collec...,4.0,1,1,0,1,0,1,4.0,1,...,0,0,0,0,0,0,0,0,0,0
3,https://www.sephora.com/product/invisimatte-bl...,4.5,0,0,0,0,0,0,10.0,1,...,0,0,0,0,0,0,0,0,0,0
4,https://www.sephora.com/product/summer-fridays...,4.5,1,1,0,1,0,1,42.0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,https://www.sephora.com/product/epidermal-re-t...,4.5,0,0,0,0,0,1,41.0,1,...,0,0,0,0,0,0,0,0,0,0
1172,https://www.sephora.com/product/coconut-rose-t...,4.5,1,1,0,1,0,1,25.0,1,...,0,0,0,0,0,0,0,0,0,0
1173,https://www.sephora.com/product/rose-hibiscus-...,4.0,1,1,0,1,0,1,32.0,1,...,0,0,0,0,0,0,0,0,0,0
1174,https://www.sephora.com/product/goop-g-tox-mal...,4.0,1,1,0,1,1,1,75.0,1,...,0,0,0,1,0,0,0,0,0,0


### Train Test Split

In [33]:
train, test = train_test_split(cleaned_data, test_size=0.2)

In [35]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [38]:
train.to_csv("sephora_train_data.csv")
test.to_csv("sephora_test_data.csv")